In [1]:
import pandas as pd
import numpy as np
import warnings 
import os
import holidays
import datetime
from datetime import timedelta



from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')


In [472]:
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### 날짜 피쳐 생성

In [473]:
#pip install holidays
# 파이썬 공휴일 함수?

In [474]:
# https://github.com/dr-prodigy/python-holidays

In [475]:
#일반 평일 : 0 / 공휴일 1일짜리 전 : 1 / 공휴일 2일짜리 전 : 2  이런 식으로 하면 오히려 점수 떨어짐 하루만 따지는게 점수 더 나음

In [476]:
holiday_list = [] 
for i in range(2016,2022):
    for date in holidays.KR(years=i).keys():  # 한국 공휴일 가져오기
        holiday_list.append(date)

In [477]:
holiday_before_list=[]
for_mat='%Y-%m-%d'
for i in holiday_list:
    holiday_before_list.append((i - timedelta(days=1)).strftime(for_mat))
        # i - timedelta(days=1): datetime에서 날짜 하루 빼기
        # .strftime(for_mat): 포맷과 같은 형식으로 datetime에서 str로 변환

In [478]:
def holiday_before(x):
    if x in holiday_before_list:
        return 1
    else:
        return 0
train['공휴일전'] = train.일자.apply(holiday_before)
test['공휴일전'] = test.일자.apply(holiday_before)

In [479]:
holiday_after_list=[]
for_mat='%Y-%m-%d'
for i in holiday_list:
    holiday_after_list.append((i + timedelta(days=1)).strftime(for_mat))

In [480]:
def holiday_after(x):
    if x in holiday_after_list:
        return 1
    else:
        return 0
train['공휴일후'] = train.일자.apply(holiday_after)
test['공휴일후'] = test.일자.apply(holiday_after)

In [481]:
## 누락된 것은 찾아서 넣음

# 16-4-13 국회의원 선거
train['공휴일전'][47]=1
train['공휴일후'][48]=1

# 17-5-9 대통령 선거
train['공휴일전'][311]=1
train['공휴일후'][312]=1

# 18-6-13 지방선거
train['공휴일전'][579]=1
train['공휴일후'][580]=1

# 20-4-15 국회의원 선거
train['공휴일전'][1027]=1
train['공휴일후'][1028]=1

# 20-9-30 추석
train['공휴일전'][1129]=1


In [482]:
train.일자 = train.일자.astype('datetime64')
test.일자 = test.일자.astype('datetime64')

In [483]:
train['월'] = train.일자.dt.month
train['일'] = train.일자.dt.day
train['년'] = train.일자.dt.year

test['월'] = test.일자.dt.month
test['일'] = test.일자.dt.day
test['년'] = test.일자.dt.year

In [484]:
train['식사가능자수'] = train['본사정원수'] - train['본사출장자수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사출장자수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']

train['출장비율'] = train['본사출장자수']/train['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']

train['야근비율'] = train['본사시간외근무명령서승인건수']/train['식사가능자수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['식사가능자수']

train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

In [485]:
def dayofweek1(x):
    if '월' in x:
        return 5
    elif "화" in x:
        return 4
    elif "수" in x:
        return 3
    elif "목" in x:
        return 2
    else:
        return 1

train['요일_중식'] = train.요일.apply(dayofweek1)
test['요일_중식'] = test.요일.apply(dayofweek1)

In [486]:
def dayofweek2(x):
    if '월' in x:
        return 5
    elif "화" in x:
        return 4
    elif "수" in x:
        return 1
    elif "목" in x:
        return 3
    else:
        return 2

train['요일_석식'] = train.요일.apply(dayofweek2)
test['요일_석식'] = test.요일.apply(dayofweek2)

In [487]:
train.loc[train.석식계==0,['일자','요일','석식메뉴']]
# 마지막주(4주차도 있고 5주차도 있는거보면 애매) 수요일
# 17-9-29: 추석 전 금요일
# 18-2-14: 설날 전 수요일
#18-9-19: 다음주 수요일이 추석이라서 마지막주 수요일이긴 함
#19-12-31: 마지막주 수요일이 성탄절, 다음날이 설날이라서 이때 안하는듯

,일자,요일,석식메뉴
204,2016-11-30,수,*
224,2016-12-28,수,*
244,2017-01-25,수,*
262,2017-02-22,수,*
281,2017-03-22,수,*
306,2017-04-26,수,*
327,2017-05-31,수,자기계발의날
346,2017-06-28,수,*자기계발의날*
366,2017-07-26,수,가정의날
392,2017-09-01,금,*


In [488]:
train['석식메뉴'][204]="없음 없음 없음 없음"
train['석식메뉴'][224]="없음 없음 없음 없음"
train['석식메뉴'][244]="없음 없음 없음 없음"
train['석식메뉴'][262]="없음 없음 없음 없음"
train['석식메뉴'][281]="없음 없음 없음 없음"
train['석식메뉴'][306]="없음 없음 없음 없음"
train['석식메뉴'][327]="없음 없음 없음 없음"
train['석식메뉴'][346]="없음 없음 없음 없음"
train['석식메뉴'][366]="없음 없음 없음 없음"
train['석식메뉴'][392]="없음 없음 없음 없음"
train['석식메뉴'][410]="없음 없음 없음 없음"
train['석식메뉴'][412]="없음 없음 없음 없음"
train['석식메뉴'][424]="없음 없음 없음 없음"
train['석식메뉴'][449]="없음 없음 없음 없음"
train['석식메뉴'][468]="없음 없음 없음 없음"
train['석식메뉴'][492]="없음 없음 없음 없음"
train['석식메뉴'][502]="없음 없음 없음 없음"
train['석식메뉴'][510]="없음 없음 없음 없음"
train['석식메뉴'][529]="없음 없음 없음 없음"
train['석식메뉴'][549]="없음 없음 없음 없음"
train['석식메뉴'][571]="없음 없음 없음 없음"
train['석식메뉴'][589]="없음 없음 없음 없음"
train['석식메뉴'][609]="없음 없음 없음 없음"
train['석식메뉴'][633]="없음 없음 없음 없음"
train['석식메뉴'][648]="없음 없음 없음 없음"
train['석식메뉴'][667]="없음 없음 없음 없음"
train['석식메뉴'][687]="없음 없음 없음 없음"
train['석식메뉴'][706]="없음 없음 없음 없음"
train['석식메뉴'][730]="없음 없음 없음 없음"
train['석식메뉴'][747]="없음 없음 없음 없음"
train['석식메뉴'][766]="없음 없음 없음 없음"
train['석식메뉴'][786]="없음 없음 없음 없음"
train['석식메뉴'][809]="없음 없음 없음 없음"
train['석식메뉴'][828]="없음 없음 없음 없음"
train['석식메뉴'][853]="없음 없음 없음 없음"
train['석식메뉴'][872]="없음 없음 없음 없음"
train['석식메뉴'][890]="없음 없음 없음 없음"
train['석식메뉴'][912]="없음 없음 없음 없음"
train['석식메뉴'][932]="없음 없음 없음 없음"
train['석식메뉴'][955]="없음 없음 없음 없음"
train['석식메뉴'][973]="없음 없음 없음 없음"
train['석식메뉴'][993]="없음 없음 없음 없음"
train['석식메뉴'][1166]="없음 없음 없음 없음"
train['석식메뉴'][586]="없음 없음 없음 없음"

## 메뉴정리(중식)

In [489]:
lunch = []
for day in range(len(train)):
    tmp = train.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch.append(tmp) 

In [490]:
bob=[]
guk=[]
banchan1=[]

for i in range(len(train)):
    if lunch[i][0]:
        bob.append(lunch[i][0])
    if lunch[i][1]:
        guk.append(lunch[i][1])
    if lunch[i][2]:
        banchan1.append(lunch[i][2])

In [491]:
train['밥_중식'] = bob
train['국_중식'] = guk
train['반찬_중식'] = banchan1

In [451]:
def bob_handle(x):
    if '쌀밥' in x:
        return "밥"
    elif "카레" in x:
        return '카레'
    elif "비빔" in x:
        return '비빔밥'
    elif ("볶음" in x) or ("필라프" in x) or ("오므라이스" in x):
        return '볶음밥'    
    elif "곤드레" in x:
        return '곤드레밥' 
    elif "콩나물" in x:
        return '콩나물밥' 
    elif "영양밥" in x:
        return '영양밥'
    elif "오곡밥" in x:
        return '밥'
    elif "덮밥" in x:
        return '덮밥'
    elif "짜장" in x:
        return '짜장밥'
    else:
        return x

train.밥_중식 = train.밥_중식.apply(bob_handle)

In [452]:
def guk_handle(x):
    if ('된장' in x) or ('냉이' in x) or ('배추' in x) or ('쑥' in x) or ("아욱" in x) or ("근대" in x):
        return "된장국"  
    elif ("김치" in x) or ("김칫" in x):
        return '김치국'  
    elif "찌개" in x:
        return '찌개'
    elif "맑은" in x:
        return '맑은국'
    elif "해장" in x:
        return '해장국' 
    elif ("장국" in x) or ("미소" in x):
        return '미소국'
    elif "스프" in x:
        return '스프'    
    elif ("만두" in x) or ("만둣" in x):
        return '만두국' 
    elif ("소고기" in x) or ("쇠고기" in x):
        return '고깃국'
    elif "떡국" in x:
        return '떡국'
    elif "냉국" in x:
        return '냉국'
    elif ("북어" in x) or ('북엇' in x) or ('황태' in x):
        return '북어국' 
    elif "미역" in x:
        return '미역국' 
    elif "계란" in x:
        return '계란국' 
    elif "들깨국" in x:
        return '들깨국'
    elif "두부국" in x:
        return '두부국' 
    elif "버섯" in x:
        return '버섯국'
    elif "어묵" in x:
        return '어묵국'
    elif "육개장" in x:
        return '육개장' 
    elif "콩나물" in x:
        return '콩나물국'
    elif ("국수" in x) or ("우동" in x) :
        return '면'
    elif "갈비탕" in x:
        return '갈비탕'  
    elif "국밥" in x:
        return '국밥'
    elif ("닭" in x) or ("삼계탕" in x) or ("백숙" in x):
        return '닭국'
    elif "탕" in x:
        return '탕'
    elif "국" in x:
        return '국'
    else:
        return x

train.국_중식 = train.국_중식.apply(guk_handle)

In [453]:
def banchan1_handle(x):
    if '오징어' in x:
        return "오징어"  
    elif "닭" in x:
        return '닭고기'
    elif "오리" in x:
        return '오리고기'
    elif "불고기" in x:
        return '불고기' 
    elif "까스" in x:
        return '돈가스'
    elif "가스" in x:
        return '돈가스'    
    elif "치킨" in x:
        return '치킨' 
    elif "장조림" in x:
        return '장조림'   
    elif "강정" in x:
        return '강정' 
    elif ("고등어" in x) or ("삼치" in x) or ("갈치" in x) or ("굴비" in x) or ("가자미" in x):
        return '생선'
    elif "낙지" in x:
        return '낙지'  
    elif "주꾸미" in x:
        return '주꾸미' 
    elif ("돼지갈비" in x) or ("소갈비" in x):
        return '갈비'
    elif "스테이크" in x:
        if "함박" in x:
            return '함박스테이크'
        else:
            return '스테이크'
    elif "함박" in x:
        return '함박스테이크'
    elif ("돈육" in x) or ("육" in x) or ("삼겹살" in x) or ("돼지" in x) or ("보쌈" in x):
        return '돼지고기'
    elif "순대" in x:
        return '순대'
    elif "탕수" in x:
        return '탕수'
    elif "새우" in x:
        return '새우'
    else:
        return x

train. 반찬_중식 = train.반찬_중식.apply(banchan1_handle)

In [492]:
lunch_test = []
for day in range(len(test)):
    tmp = test.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch_test.append(tmp) 

bob_test=[]
guk_test=[]
banchan1_test=[]

for i in range(len(test)):
    if lunch_test[i][0]:
        bob_test.append(lunch_test[i][0])
    if lunch_test[i][1]:
        guk_test.append(lunch_test[i][1])
    if lunch_test[i][2]:
        banchan1_test.append(lunch_test[i][2])

test['밥_중식'] = bob_test
test['국_중식'] = guk_test
test['반찬_중식'] = banchan1_test

In [455]:
test.밥_중식 = test.밥_중식.apply(bob_handle)
test.국_중식 = test.국_중식.apply(guk_handle)
test. 반찬_중식 = test.반찬_중식.apply(banchan1_handle)

## 메뉴정리(저녁)

In [493]:
dinner = []
for day in range(len(train)):
    tmp = train.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner.append(tmp) 

In [494]:
bob=[]
guk=[]
banchan=[]

for i in range(len(train)):
    if dinner[i][0]:
        bob.append(dinner[i][0])
    if dinner[i][1]:
        guk.append(dinner[i][1])
    if dinner[i][2]:
        banchan.append(dinner[i][2])

In [495]:
train['밥_석식'] = bob
train['국_석식'] = guk
train['반찬_석식'] = banchan

In [459]:
train.밥_석식 = train.밥_석식.apply(bob_handle)
train.국_석식 = train.국_석식.apply(guk_handle)
train. 반찬_석식 = train.반찬_석식.apply(banchan1_handle)

In [496]:
dinner_test = []
for day in range(len(test)):
    tmp = test.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner_test.append(tmp) 

bob_test=[]
guk_test=[]
banchan1_test=[]

for i in range(len(test)):
    if dinner_test[i][0]:
        bob_test.append(dinner_test[i][0])
    if dinner_test[i][1]:
        guk_test.append(dinner_test[i][1])
    if dinner_test[i][2]:
        banchan1_test.append(dinner_test[i][2])

test['밥_석식'] = bob_test
test['국_석식'] = guk_test
test['반찬_석식'] = banchan1_test

In [461]:
test.밥_석식 = test.밥_석식.apply(bob_handle)
test.국_석식 = test.국_석식.apply(guk_handle)
test. 반찬_석식 = test.반찬_석식.apply(banchan1_handle)

## 모델링 준비

In [497]:
X1_train = train.drop(['일자','요일', '본사정원수', '본사휴가자수', '본사출장자수',
                       '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
                       '년','조식메뉴','중식메뉴',
                       '석식메뉴','중식계','석식계','요일_석식',
                      '밥_석식','국_석식','반찬_석식'], axis=1)
X2_train = train.drop(['일자','요일', '본사정원수', '본사휴가자수', '본사출장자수',
                       '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
                       '년','조식메뉴','중식메뉴',
                       '석식메뉴','중식계','석식계','요일_중식',
                      '밥_중식','국_중식','반찬_중식'], axis=1)
y1_train = train.중식계
y2_train = train.석식계

X1_test = test.drop(['일자','요일', '본사정원수', '본사휴가자수', '본사출장자수',
                    '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
                    '년','조식메뉴','중식메뉴','석식메뉴', '요일_석식',
                    '밥_석식','국_석식','반찬_석식'],axis=1)
X2_test = test.drop(['일자','요일', '본사정원수', '본사휴가자수', '본사출장자수',
                    '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
                    '년','조식메뉴','중식메뉴','석식메뉴','요일_중식',
                    '밥_중식','국_중식','반찬_중식'],axis=1)

In [498]:
#index_0 = train[train['석식계']==0].index
#X2_train = X2_train.drop(index_0)
#y2_train = y2_train.drop(index_0)

## 석식계가 0인 행을 지우면 점수가 더 떨어진다. 내가한거 기준

In [499]:
#print(X2_train.shape,y2_train.shape)

In [500]:
cat_features=['밥_중식','국_중식','반찬_중식']
features = pd.concat([X1_train, X1_test]).reset_index(drop=True)

for i in cat_features:
    features[i] = LabelEncoder().fit_transform(features[i])
    
X1_train = features[:X1_train.shape[0]]
X1_test = features[X1_train.shape[0]:]

In [501]:
cat_features2=['밥_석식','국_석식','반찬_석식']
features2 = pd.concat([X2_train, X2_test]).reset_index(drop=True)

for i in cat_features2:
    features2[i] = LabelEncoder().fit_transform(features2[i])
    
X2_train = features2[:X2_train.shape[0]]
X2_test = features2[X2_train.shape[0]:]

In [502]:
model1 = LGBMRegressor(random_state=42)
model2 = LGBMRegressor(random_state=42)

In [503]:
model1.fit(X1_train, y1_train)

LGBMRegressor(random_state=42)

In [504]:
model2.fit(X2_train, y2_train)

LGBMRegressor(random_state=42)

In [505]:
pred1 = model1.predict(X1_test)
pred2 = model2.predict(X2_test)

In [506]:
sample['중식계'] = pred1
sample['석식계'] = pred2

sample.to_csv('test1.csv', index=False)

In [ ]:
# 퍼블릭: 77.0141411973
# 프라이빗: 117.3859862906
